In [1]:
!pip install dagshub --upgrade
!pip install -q pandas numpy scikit-learn mlflow joblib kaggle
!pip install torch plotly tqdm --quiet
!pip install statsmodels --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.2/261.2 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.9/13.9 MB 68.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.3/85.3 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.3/74.3 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.7/24.7 MB 70.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 64.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8

In [159]:
%pip install -q dagshub 'mlflow>=2,<3'

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.0/29.0 MB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 6.1 MB/s eta 0:00:00


In [163]:
import dagshub
dagshub.init(repo_owner='jgushiann', repo_name='Walmart-Recruiting---Store-Sales-Forecasting', mlflow=True)

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

Output()



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=de3d4283-ebf1-4bcd-bd58-13072bba7370&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=ca8ba97fd4226e0359cd25202fc4bf40c075d165da201cea5bc38b41c1dfd71b




Accessing as jgushiann

Initialized MLflow to track repo "jgushiann/Walmart-Recruiting---Store-Sales-Forecasting"

Repository jgushiann/Walmart-Recruiting---Store-Sales-Forecasting initialized!

In [161]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import joblib
import warnings
import os
import zipfile
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

warnings.filterwarnings("ignore")
np.random.seed(42)
torch.manual_seed(42)

In [128]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [129]:
SEQ_LEN = 4
PRED_LEN = 2
BATCH_SIZE = 64
EPOCHS = 50
LEARNING_RATE = 0.001
PATIENCE = 10
TEST_SIZE = 0.1
VAL_SIZE = 0.05

In [130]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [131]:
from google.colab import files
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [132]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle competitions download -c walmart-recruiting-store-sales-forecasting

!mkdir -p data
!unzip walmart-recruiting-store-sales-forecasting.zip -d data/

walmart-recruiting-store-sales-forecasting.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  walmart-recruiting-store-sales-forecasting.zip
  inflating: data/features.csv.zip   
  inflating: data/sampleSubmission.csv.zip  
replace data/stores.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: data/stores.csv         
  inflating: data/test.csv.zip       
  inflating: data/train.csv.zip      


In [133]:
import os
print(os.listdir("data"))

['features.csv.zip', 'stores.csv', 'test.csv', 'train.csv.zip', 'sampleSubmission.csv', 'features.csv', 'test.csv.zip', 'sampleSubmission.csv.zip', 'train.csv']


In [134]:
import zipfile

data_dir = "data"
for file in os.listdir(data_dir):
    if file.endswith(".zip"):
        file_path = os.path.join(data_dir, file)
        with zipfile.ZipFile(file_path, 'r') as zip_ref:
            zip_ref.extractall(data_dir)
        print(f"Unzipped {file}")

print("Updated contents of data directory:", os.listdir(data_dir))

!rm data/*.zip
print("Remaining files:", os.listdir("data"))

Unzipped features.csv.zip
Unzipped train.csv.zip
Unzipped test.csv.zip
Unzipped sampleSubmission.csv.zip
Updated contents of data directory: ['features.csv.zip', 'stores.csv', 'test.csv', 'train.csv.zip', 'sampleSubmission.csv', 'features.csv', 'test.csv.zip', 'sampleSubmission.csv.zip', 'train.csv']
Remaining files: ['stores.csv', 'test.csv', 'sampleSubmission.csv', 'features.csv', 'train.csv']


In [135]:
train = pd.read_csv("data/train.csv")
stores = pd.read_csv("data/stores.csv")
features = pd.read_csv("data/features.csv")
test = pd.read_csv("data/test.csv")

print("Train shape:", train.shape)
print("Stores shape:", stores.shape)
print("Features shape:", features.shape)
print("Test shape:", test.shape)

Train shape: (421570, 5)
Stores shape: (45, 3)
Features shape: (8190, 12)
Test shape: (115064, 4)


In [136]:
train_full = train.merge(stores, on="Store").merge(features, on=["Store", "Date"], how="left")
train_full["Date"] = pd.to_datetime(train_full["Date"])

test_full = test.merge(stores, on='Store').merge(features, on=['Store', 'Date'], how="left")
test_full["Date"] = pd.to_datetime(test_full["Date"])

In [137]:
if 'IsHoliday_x' in train_full.columns:
    train_full['IsHoliday'] = train_full['IsHoliday_x']
    train_full = train_full.drop(columns=['IsHoliday_x', 'IsHoliday_y'], errors='ignore')

if 'IsHoliday_x' in test_full.columns:
    test_full['IsHoliday'] = test_full['IsHoliday_x']
    test_full = test_full.drop(columns=['IsHoliday_x', 'IsHoliday_y'], errors='ignore')

print(f"Train shape: {train_full.shape}, Test shape: {test_full.shape}")

Train shape: (421570, 16), Test shape: (115064, 15)


In [138]:
def add_time_features(df):
    df = df.copy()
    df["Year"] = df["Date"].dt.year
    df["Month"] = df["Date"].dt.month
    df["Week"] = df["Date"].dt.isocalendar().week
    df["Quarter"] = df["Date"].dt.quarter
    df["IsHoliday"] = df["IsHoliday"].astype(int)

    df["Month_sin"] = np.sin(2 * np.pi * df["Month"] / 12)
    df["Month_cos"] = np.cos(2 * np.pi * df["Month"] / 12)
    df["Week_sin"] = np.sin(2 * np.pi * df["Week"] / 52)
    df["Week_cos"] = np.cos(2 * np.pi * df["Week"] / 52)

    df["Season"] = df["Month"].map({12: 4, 1: 4, 2: 4, 3: 1, 4: 1, 5: 1,
                                   6: 2, 7: 2, 8: 2, 9: 3, 10: 3, 11: 3})
    return df

In [139]:
def add_lag_features(df, target_col='Weekly_Sales'):
    df = df.copy()
    df = df.sort_values(['Store', 'Dept', 'Date'])

    for lag in [1, 2, 4, 8, 12]:
        df[f'{target_col}_lag_{lag}'] = df.groupby(['Store', 'Dept'])[target_col].shift(lag)

    for window in [4, 8, 12]:
        df[f'{target_col}_rolling_mean_{window}'] = df.groupby(['Store', 'Dept'])[target_col].transform(lambda x: x.rolling(window).mean())
        df[f'{target_col}_rolling_std_{window}'] = df.groupby(['Store', 'Dept'])[target_col].transform(lambda x: x.rolling(window).std())

    return df

In [140]:
def handle_missing_values(df):
    df = df.copy()

    markdown_cols = [col for col in df.columns if 'MarkDown' in col]
    for col in markdown_cols:
        df[col] = df[col].fillna(0)

    for col in ['CPI', 'Unemployment', 'Temperature', 'Fuel_Price']:
        if col in df.columns:
            df[col] = df.groupby('Store')[col].transform(lambda x: x.fillna(method='ffill').fillna(method='bfill'))
            df[col] = df[col].fillna(df[col].median())

    lag_cols = [col for col in df.columns if '_lag_' in col or '_rolling_' in col]
    for col in lag_cols:
        df[col] = df[col].fillna(0)

    return df

In [141]:
def preprocess_data(df, sales_scaler=None, feature_scaler=None, cat_encoder=None, fit_scalers=True):
    df = add_time_features(df)
    df = add_lag_features(df)
    df = handle_missing_values(df)

    numerical_features = [
        'Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'Size',
        'Month_sin', 'Month_cos', 'Week_sin', 'Week_cos',
        'Store', 'Dept'
    ] + [col for col in df.columns if 'MarkDown' in col or '_lag_' in col or '_rolling_' in col]

    categorical_features = ['Type', 'Season']

    numerical_features = [f for f in numerical_features if f in df.columns]
    categorical_features = [f for f in categorical_features if f in df.columns]

    if fit_scalers:
        sales_scaler = StandardScaler()
        feature_scaler = StandardScaler()
        cat_encoder = OneHotEncoder(sparse_output=False, drop='first', handle_unknown='ignore')

    if numerical_features:
        if fit_scalers:
            df[numerical_features] = feature_scaler.fit_transform(df[numerical_features])
        else:
            df[numerical_features] = feature_scaler.transform(df[numerical_features])

    if categorical_features:
        if fit_scalers:
            cat_encoded = cat_encoder.fit_transform(df[categorical_features])
        else:
            cat_encoded = cat_encoder.transform(df[categorical_features])

        cat_names = cat_encoder.get_feature_names_out(categorical_features)

        cat_df = pd.DataFrame(cat_encoded, columns=cat_names)

        df = df.reset_index(drop=True)
        cat_df = cat_df.reset_index(drop=True)

        df = pd.concat([df, cat_df], axis=1)
        df = df.drop(columns=categorical_features)
        numerical_features.extend(cat_names)

    if 'Weekly_Sales' in df.columns:
        if fit_scalers:
            df['Weekly_Sales_Scaled'] = sales_scaler.fit_transform(df[['Weekly_Sales']]).flatten()
        else:
            df['Weekly_Sales_Scaled'] = sales_scaler.transform(df[['Weekly_Sales']]).flatten()

    return df, numerical_features, sales_scaler, feature_scaler, cat_encoder


In [142]:
class DLinear(nn.Module):
    def __init__(self, input_size, seq_len, pred_len):
        super(DLinear, self).__init__()
        self.seq_len = seq_len
        self.pred_len = pred_len

        kernel_size = min(25, seq_len // 4)
        self.decomposition = nn.AvgPool1d(
            kernel_size, stride=1,
            padding=(kernel_size-1)//2,
            count_include_pad=False
        )

        self.Linear_Seasonal = nn.Linear(seq_len, pred_len)
        self.Linear_Trend = nn.Linear(seq_len, pred_len)

        if input_size > 0:
            self.feature_projection = nn.Sequential(
                nn.Linear(input_size, 64),
                nn.ReLU(),
                nn.Dropout(0.1),
                nn.Linear(64, pred_len)
            )
        else:
            self.feature_projection = None

    def forward(self, x, x_mark=None):
        moving_mean = self.decomposition(x.permute(0, 2, 1)).permute(0, 2, 1)
        seasonal = x - moving_mean

        seasonal = seasonal.permute(0, 2, 1)
        trend = moving_mean.permute(0, 2, 1)

        seasonal_output = self.Linear_Seasonal(seasonal)
        trend_output = self.Linear_Trend(trend)

        output = seasonal_output + trend_output

        if self.feature_projection is not None and x_mark is not None:
            feature_out = self.feature_projection(x_mark.mean(dim=1))
            output = output + feature_out.unsqueeze(1)

        return output.permute(0, 2, 1)

In [143]:
class TimeSeriesDataset(Dataset):
    def __init__(self, df, seq_len, pred_len, feature_columns, min_length=6):
        self.df = df.sort_values(['Store', 'Dept', 'Date']).reset_index(drop=True)
        self.seq_len = seq_len
        self.pred_len = pred_len
        self.feature_columns = feature_columns
        self.sequences = []

        total_groups = 0
        valid_groups = 0

        # Create sequences for each store-department combination
        for (store, dept), group in self.df.groupby(['Store', 'Dept']):
            total_groups += 1

            if len(group) >= max(min_length, seq_len + pred_len):
                group = group.sort_values('Date').reset_index(drop=True)
                valid_groups += 1

                # Create multiple sequences with stride=1 for more data
                max_start = len(group) - seq_len - pred_len + 1
                for i in range(0, max_start, 2):  # stride=2 to avoid too much overlap
                    seq_data = group.iloc[i:i + seq_len]
                    target_data = group.iloc[i + seq_len:i + seq_len + pred_len]

                    # Skip if missing values
                    if seq_data['Weekly_Sales_Scaled'].isna().any() or target_data['Weekly_Sales_Scaled'].isna().any():
                        continue

                    self.sequences.append({
                        'seq_sales': seq_data['Weekly_Sales_Scaled'].values,
                        'seq_features': seq_data[self.feature_columns].values,
                        'target_sales': target_data['Weekly_Sales_Scaled'].values
                    })


        print(f"Total groups: {total_groups}, Valid groups: {valid_groups}, Created {len(self.sequences)} sequences")

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        seq = self.sequences[idx]
        x = torch.FloatTensor(seq['seq_sales']).unsqueeze(-1)
        x_mark = torch.FloatTensor(seq['seq_features'])
        y = torch.FloatTensor(seq['target_sales']).unsqueeze(-1)
        return x, x_mark, y


In [144]:
def split_data_by_time(df, test_size=0.2, val_size=0.1):
    df_sorted = df.sort_values('Date')
    unique_dates = sorted(df_sorted['Date'].unique())

    test_dates = int(len(unique_dates) * test_size)
    val_dates = int(len(unique_dates) * val_size)

    train_end_date = unique_dates[len(unique_dates) - test_dates - val_dates - 1]
    val_end_date = unique_dates[len(unique_dates) - test_dates - 1]

    train_df = df_sorted[df_sorted['Date'] <= train_end_date]
    val_df = df_sorted[(df_sorted['Date'] > train_end_date) & (df_sorted['Date'] <= val_end_date)]
    test_df = df_sorted[df_sorted['Date'] > val_end_date]

    print(f"Train: {len(train_df)} samples, Val: {len(val_df)} samples, Test: {len(test_df)} samples")
    return train_df, val_df, test_df

In [145]:
def calculate_metrics(y_true, y_pred):
    y_true_safe = np.where(np.abs(y_true) < 1e-8, 1e-8, y_true)

    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mape = np.mean(np.abs((y_true - y_pred) / y_true_safe)) * 100
    r2 = r2_score(y_true, y_pred)

    return {'MAE': mae, 'RMSE': rmse, 'MAPE': mape, 'R2': r2}

In [146]:
def plot_results(train_losses, val_losses, predictions, targets):
    fig = go.Figure()
    epochs = list(range(1, len(train_losses) + 1))
    fig.add_trace(go.Scatter(x=epochs, y=train_losses, name='Train Loss'))
    fig.add_trace(go.Scatter(x=epochs, y=val_losses, name='Val Loss'))
    fig.update_layout(title='Training History', xaxis_title='Epoch', yaxis_title='Loss')
    fig.show()

    if len(predictions) > 0 and len(targets) > 0:
        fig = make_subplots(rows=1, cols=2, subplot_titles=['Predictions vs Actual', 'Sample Time Series'])

        fig.add_trace(go.Scatter(x=targets, y=predictions, mode='markers', name='Predictions'), row=1, col=1)
        min_val, max_val = min(targets.min(), predictions.min()), max(targets.max(), predictions.max())
        fig.add_trace(go.Scatter(x=[min_val, max_val], y=[min_val, max_val],
                                mode='lines', name='Perfect', line=dict(dash='dash')), row=1, col=1)

        sample_size = min(100, len(targets))
        sample_idx = np.random.choice(len(targets), sample_size, replace=False)
        sample_idx = np.sort(sample_idx)

        fig.add_trace(go.Scatter(y=targets[sample_idx], name='Actual'), row=1, col=2)
        fig.add_trace(go.Scatter(y=predictions[sample_idx], name='Predicted'), row=1, col=2)

        fig.update_layout(height=400, title='Model Evaluation')
        fig.show()
    else:
        print("No test predictions to plot - test dataset was empty.")
        print("Model training completed successfully, but no test evaluation possible.")


In [147]:
def train_model(model, train_loader, val_loader, sales_scaler):
    # Check if loaders have data
    if len(train_loader) == 0:
        raise ValueError("Training loader is empty! Not enough data to create sequences.")
    if len(val_loader) == 0:
        print("Warning: Validation loader is empty! Using training loss for validation.")
        use_val = False
    else:
        use_val = True

    optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-5)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=PATIENCE//2, factor=0.5)
    criterion = nn.MSELoss()

    best_val_loss = float('inf')
    patience_counter = 0
    train_losses = []
    val_losses = []

    for epoch in range(EPOCHS):
        # Training
        model.train()
        train_loss = 0
        for x, x_mark, y in tqdm(train_loader, desc=f'Epoch {epoch+1}'):
            x, x_mark, y = x.to(device), x_mark.to(device), y.to(device)

            optimizer.zero_grad()
            output = model(x, x_mark)
            loss = criterion(output, y)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()

            train_loss += loss.item()

        avg_train_loss = train_loss / len(train_loader)
        train_losses.append(avg_train_loss)

        # Validation
        if use_val:
            model.eval()
            val_loss = 0
            with torch.no_grad():
                for x, x_mark, y in val_loader:
                    x, x_mark, y = x.to(device), x_mark.to(device), y.to(device)
                    output = model(x, x_mark)
                    loss = criterion(output, y)
                    val_loss += loss.item()

            avg_val_loss = val_loss / len(val_loader)
            val_losses.append(avg_val_loss)
            scheduler.step(avg_val_loss)

            print(f'Epoch {epoch+1}: Train Loss: {avg_train_loss:.6f}, Val Loss: {avg_val_loss:.6f}')

            # Early stopping
            if avg_val_loss < best_val_loss:
                best_val_loss = avg_val_loss
                patience_counter = 0
                torch.save(model.state_dict(), 'best_model.pth')
            else:
                patience_counter += 1
                if patience_counter >= PATIENCE:
                    print('Early stopping!')
                    break
        else:
            # No validation data - use training loss
            val_losses.append(avg_train_loss)
            scheduler.step(avg_train_loss)
            print(f'Epoch {epoch+1}: Train Loss: {avg_train_loss:.6f} (No validation data)')

            if avg_train_loss < best_val_loss:
                best_val_loss = avg_train_loss
                torch.save(model.state_dict(), 'best_model.pth')

    # Load best model
    if os.path.exists('best_model.pth'):
        model.load_state_dict(torch.load('best_model.pth'))
    return train_losses, val_losses

In [148]:
def evaluate_model(model, test_loader, sales_scaler):
    if len(test_loader) == 0:
        print("Warning: Test loader is empty! Cannot evaluate model.")
        print("This means your dataset doesn't have enough consecutive data for testing.")
        print("Try reducing SEQ_LEN and PRED_LEN further, or use a different dataset.")
        return np.array([]), np.array([]), {}

    model.eval()
    all_predictions = []
    all_targets = []

    with torch.no_grad():
        for x, x_mark, y in tqdm(test_loader, desc="Evaluating"):
            x, x_mark, y = x.to(device), x_mark.to(device), y.to(device)
            output = model(x, x_mark)

            all_predictions.append(output.cpu().numpy())
            all_targets.append(y.cpu().numpy())

    if len(all_predictions) == 0:
        print("No predictions generated!")
        return np.array([]), np.array([]), {}

    predictions = np.concatenate(all_predictions, axis=0).reshape(-1)
    targets = np.concatenate(all_targets, axis=0).reshape(-1)

    # Convert back to original scale
    predictions_orig = sales_scaler.inverse_transform(predictions.reshape(-1, 1)).flatten()
    targets_orig = sales_scaler.inverse_transform(targets.reshape(-1, 1)).flatten()

    # Calculate metrics
    metrics = calculate_metrics(targets_orig, predictions_orig)

    print("Test Metrics (Original Scale):")
    for metric, value in metrics.items():
        print(f"  {metric}: {value:.4f}")

    return predictions_orig, targets_orig, metrics

In [149]:
processed_data, feature_columns, sales_scaler, feature_scaler, cat_encoder = preprocess_data(
    train_full, fit_scalers=True
)

In [150]:
train_df, val_df, test_df = split_data_by_time(processed_data, TEST_SIZE, VAL_SIZE)

Train: 359432 samples, Val: 20675 samples, Test: 41463 samples


In [151]:
min_seq_len = SEQ_LEN + PRED_LEN  # e.g., 36 + 24 = 60 or your adjusted values

def filter_short_groups(df, min_len):
    group_sizes = df.groupby(['Store', 'Dept']).size()
    valid_groups = group_sizes[group_sizes >= min_len].index
    filtered_df = df.set_index(['Store', 'Dept']).loc[valid_groups].reset_index()
    return filtered_df

train_df_filtered = filter_short_groups(train_df, min_seq_len)
val_df_filtered = filter_short_groups(val_df, min_seq_len)
test_df_filtered = filter_short_groups(test_df, min_seq_len)

print("Filtered lengths:")
print("Train min group length:", train_df_filtered.groupby(['Store','Dept']).size().min())
print("Val min group length:", val_df_filtered.groupby(['Store','Dept']).size().min())
print("Test min group length:", test_df_filtered.groupby(['Store','Dept']).size().min())

Filtered lengths:
Train min group length: 6
Val min group length: 6
Test min group length: 6


In [152]:
train_dataset = TimeSeriesDataset(train_df_filtered, SEQ_LEN, PRED_LEN, feature_columns)
val_dataset = TimeSeriesDataset(val_df_filtered, SEQ_LEN, PRED_LEN, feature_columns)
test_dataset = TimeSeriesDataset(test_df_filtered, SEQ_LEN, PRED_LEN, feature_columns)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

Total groups: 3201, Valid groups: 3201, Created 173021 sequences
Total groups: 2888, Valid groups: 2888, Created 2888 sequences
Total groups: 2999, Valid groups: 2999, Created 14570 sequences


In [153]:
sample_x, sample_x_mark, sample_y = next(iter(train_loader))
print(f"Sample shapes - x: {sample_x.shape}, x_mark: {sample_x_mark.shape}, y: {sample_y.shape}")
print(f"Feature columns count: {len(feature_columns)}")
print(f"Actual x_mark features: {sample_x_mark.shape[-1]}")

input_size = sample_x_mark.shape[-1]  # Use actual feature dimension
model = DLinear(input_size, SEQ_LEN, PRED_LEN).to(device)
print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")

Sample shapes - x: torch.Size([64, 4, 1]), x_mark: torch.Size([64, 4, 32]), y: torch.Size([64, 2, 1])
Feature columns count: 32
Actual x_mark features: 32
Model parameters: 2,262


In [164]:
import mlflow
with mlflow.start_run():
  train_losses, val_losses = train_model(model, train_loader, val_loader, sales_scaler)
  mlflow.log_metric('accuracy', 42)
  mlflow.log_param('Param name', 'Value')

Epoch 1: 100%|██████████| 2704/2704 [00:13<00:00, 193.16it/s]


Epoch 1: Train Loss: 0.063821, Val Loss: 0.031067


Epoch 2: 100%|██████████| 2704/2704 [00:13<00:00, 194.02it/s]


Epoch 2: Train Loss: 0.062450, Val Loss: 0.028652


Epoch 3: 100%|██████████| 2704/2704 [00:17<00:00, 152.74it/s]


...

Epoch 21: Train Loss: 0.051835, Val Loss: 0.024954


Epoch 22: 100%|██████████| 2704/2704 [00:13<00:00, 193.28it/s]


Epoch 22: Train Loss: 0.052073, Val Loss: 0.025218


Epoch 23: 100%|██████████| 2704/2704 [00:13<00:00, 196.19it/s]


Epoch 23: Train Loss: 0.051554, Val Loss: 0.023785
Early stopping!
🏃 View run adorable-hog-400 at: https://dagshub.com/jgushiann/Walmart-Recruiting---Store-Sales-Forecasting.mlflow/#/experiments/0/runs/7b23c5e4c9eb43c08c724cf4f448cc9e
🧪 View experiment at: https://dagshub.com/jgushiann/Walmart-Recruiting---Store-Sales-Forecasting.mlflow/#/experiments/0


In [165]:
predictions, targets, metrics = evaluate_model(model, test_loader, sales_scaler)

Evaluating: 100%|██████████| 228/228 [00:00<00:00, 342.11it/s]

Test Metrics (Original Scale):
  MAE: 2430.1650
  RMSE: 4001.7834
  MAPE: 10758.0645
  R2: 0.9666


In [166]:
plot_results(train_losses, val_losses, predictions, targets)

torch.save(model.state_dict(), 'final_model.pth')
joblib.dump({'sales_scaler': sales_scaler, 'feature_scaler': feature_scaler,
             'cat_encoder': cat_encoder, 'feature_columns': feature_columns}, 'scalers.pkl')

['scalers.pkl']

In [169]:
print(test_full)

        Store  Dept       Date Type    Size  Temperature  Fuel_Price  \
0           1     1 2012-11-02    A  151315        55.32       3.386   
1           1     1 2012-11-09    A  151315        61.24       3.314   
2           1     1 2012-11-16    A  151315        52.92       3.252   
3           1     1 2012-11-23    A  151315        56.23       3.211   
4           1     1 2012-11-30    A  151315        52.34       3.207   
...       ...   ...        ...  ...     ...          ...         ...   
115059     45    98 2013-06-28    B  118221        76.05       3.639   
115060     45    98 2013-07-05    B  118221        77.50       3.614   
115061     45    98 2013-07-12    B  118221        79.37       3.614   
115062     45    98 2013-07-19    B  118221        82.84       3.737   
115063     45    98 2013-07-26    B  118221        76.06       3.804   

        MarkDown1  MarkDown2  MarkDown3  MarkDown4  MarkDown5         CPI  \
0         6766.44    5147.70      50.82    3639.90    2737

In [167]:
def predict_new_data(new_df, model_path='final_model.pth', scalers_path='scalers.pkl'):
    scalers = joblib.load(scalers_path)
    sales_scaler = scalers['sales_scaler']
    feature_scaler = scalers['feature_scaler']
    cat_encoder = scalers['cat_encoder']
    feature_columns = scalers['feature_columns']

    model = DLinear(len(feature_columns), SEQ_LEN, PRED_LEN).to(device)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()

    processed_new, _, _, _, _ = preprocess_data(
        new_df, sales_scaler, feature_scaler, cat_encoder, fit_scalers=False
    )

    new_dataset = TimeSeriesDataset(processed_new, SEQ_LEN, PRED_LEN, feature_columns)
    new_loader = DataLoader(new_dataset, batch_size=BATCH_SIZE, shuffle=False)

    predictions = []
    with torch.no_grad():
        for x, x_mark, _ in new_loader:
            x, x_mark = x.to(device), x_mark.to(device)
            output = model(x, x_mark)
            predictions.append(output.cpu().numpy())

    predictions = np.concatenate(predictions, axis=0)
    predictions_flat = predictions.reshape(-1, 1)
    predictions_original = sales_scaler.inverse_transform(predictions_flat)

    return predictions_original.reshape(predictions.shape[:-1])

In [168]:
def generate_submission(test_df, model_path="final_model.pth", scalers_path="scalers.pkl"):
    predictions_array = predict_new_data(test_df, model_path, scalers_path)

    submission = test_df[['Store', 'Dept', 'Date']].copy()

    submission = submission.sort_values(['Store', 'Dept', 'Date']).reset_index(drop=True)

    pred_len = PRED_LEN
    submission = submission.groupby(['Store', 'Dept']).head(pred_len).reset_index(drop=True)

    submission['Weekly_Sales'] = predictions_array[:len(submission)]

    return submission
